<a href="https://colab.research.google.com/github/hqphuoc129/UIT-AI-Challenge2020/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Top1: BaiDu Team

Method Used: Tracking by Detection



#Detection method: Paddle Detection

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Demo Paddle Detection

IPYN Demo
[link text](https://github.com/PaddlePaddle/PaddleDetection/blob/release/0.2/demo/mask_rcnn_demo.ipynb)

![alt text](https://raw.githubusercontent.com/PaddlePaddle/PaddleDetection/release/0.4/docs/images/000000570688.jpg)

#Tracking Method: modified DeepSort 
Prepare format for MOT:
[link text](https://github.com/PaddlePaddle/Research/blob/master/CV/VehicleCounting/utils/prepare_for_mot.py)

DeepSort Colab Tutorial:
[link text](https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/DeepSORT_YOLOv3.ipynb#scrollTo=s-swJb8AnUwf)



#Counting 

Code for calculating Distances

In [ ]:
import numba
import numpy as np
from math import sqrt, pow, cos, sin, asin

@numba.jit(nopython=True, fastmath=True)
def manhattan(array_x, array_y):
	n = array_x.shape[0]
	ret = 0.
	for i in range(n):
		ret += abs(array_x[i]-array_y[i])
	return ret

@numba.jit(nopython=True, fastmath=True)
def euclidean(array_x, array_y):
	n = array_x.shape[0]
	ret = 0.
	for i in range(n):
		ret += (array_x[i]-array_y[i])**2
	return sqrt(ret)

@numba.jit(nopython=True, fastmath=True)
def chebyshev(array_x, array_y):
	n = array_x.shape[0]
	ret = -1*np.inf
	for i in range(n):
		d = abs(array_x[i]-array_y[i])
		if d>ret: ret=d
	return ret

@numba.jit(nopython=True, fastmath=True)
def cosine(array_x, array_y):
	n = array_x.shape[0]
	xy_dot = 0.
	x_norm = 0.
	y_norm = 0.
	for i in range(n):
		xy_dot += array_x[i]*array_y[i]
		x_norm += array_x[i]*array_x[i]
		y_norm += array_y[i]*array_y[i]
	return 1.-xy_dot/(sqrt(x_norm)*sqrt(y_norm))

@numba.jit(nopython=True, fastmath=True)
def haversine(array_x, array_y):
	R = 6378.0
	radians = np.pi / 180.0
	lat_x = radians * array_x[0]
	lon_x = radians * array_x[1]
	lat_y = radians * array_y[0]
	lon_y = radians * array_y[1]
	dlon = lon_y - lon_x
	dlat = lat_y - lat_x
	a = (pow(sin(dlat/2.0), 2.0) + cos(lat_x) *
		cos(lat_y) * pow(sin(dlon/2.0), 2.0))
	return R * 2 * asin(sqrt(a))

Hausdoff Disance

In [ ]:
import numpy as np
import numba
import distances
from inspect import getmembers

def _find_available_functions(module_name):
    all_members = getmembers(module_name)
    available_functions = [member[0] for member in all_members if isinstance(member[1], numba.targets.registry.CPUDispatcher)]
    return available_functions

@numba.jit(nopython=True, fastmath=True)
def _hausdorff(XA, XB, distance_function):
    nA = XA.shape[0]
    nB = XB.shape[0]
    cmax = 0.
    for i in range(nA):
        cmin = np.inf
        for j in range(nB):
            d = distance_function(XA[i,:], XB[j,:])
            if d<cmin:
                cmin = d
            if cmin<cmax:
                break
        if cmin>cmax and np.inf>cmin:
            cmax = cmin
    '''
	for j in range(nB):
		cmin = np.inf
		for i in range(nA):
			d = distance_function(XA[i,:], XB[j,:])
			if d<cmin:
				cmin = d
			if cmin<cmax:
				break
		if cmin>cmax and np.inf>cmin:
			cmax = cmin
    '''
    return cmax

def hausdorff_distance(XA, XB, distance='euclidean'):
    assert distance in _find_available_functions(distances), \
        'distance is not an implemented function'
    assert type(XA) is np.ndarray and type(XB) is np.ndarray, \
        'arrays must be of type numpy.ndarray'
    assert np.issubdtype(XA.dtype, np.number) and np.issubdtype(XA.dtype, np.number), \
        'the arrays data type must be numeric'
    assert XA.ndim == 2 and XB.ndim == 2, \
        'arrays must be 2-dimensional'
    assert XA.shape[1] == XB.shape[1], \
        'arrays must have equal number of columns'
    if distance == 'haversine':
        assert XA.shape[1] >= 2, 'haversine distance requires at least 2 coordinates per point (lat, lng)'
        assert XB.shape[1] >= 2, 'haversine distance requires at least 2 coordinates per point (lat, lng)'
    distance_function = getattr(distances, distance)
    return _hausdorff(XA, XB, distance_function)

Main code Counting

In [ ]:
import os
import sys
import json
import cv2
import numpy as np
from hausdorff_dist import hausdorff_distance

def check_bbox_inside_with_roi(bbox, mask):
    #check if four point of bbox all in roi area
    is_inside = True

    x_tl = bbox[1]
    y_tl = bbox[2]
    x_br = bbox[3]
    y_br = bbox[4]

    for x in [x_tl, x_br]:
        if x <= 0 or x >= mask.shape[1]:
            return False

    for y in [y_tl, y_br]:
        if y <= 0 or y >= mask.shape[0]:
            return False

    vertexs = [[x_tl, y_tl], [x_tl, y_br], [x_br, y_tl], [x_br, y_br]]
    for v in vertexs:
        (g, b, r) = mask[v[1], v[0]]
        if (g, b, r) < (128, 128, 128):
            is_inside = False
            return is_inside

    return is_inside

def check_tracks_with_roi(tracks, mask):
    tracks_end_in_roi = []
    tracks_start_in_roi = []
    tracks_too_short = []

    for trackid, track in tracks.items():
        start_bbox = track['bbox'][0]
        end_bbox = track['bbox'][-1]

        if check_bbox_inside_with_roi(start_bbox, mask) == True:
            if track['startframe'] > 3:
                tracks_start_in_roi.append(trackid)

        if check_bbox_inside_with_roi(end_bbox, mask) == True:
            tracks_end_in_roi.append(trackid)

        if track['endframe'] - track['startframe'] < 10:
            if trackid not in tracks_start_in_roi:
                tracks_too_short.append(trackid)
    return tracks_end_in_roi, tracks_start_in_roi, tracks_too_short


def check_bbox_overlap_with_roi(bbox, mask):
    is_overlap = False
    if bbox[1] >= mask.shape[1] or bbox[2] >= mask.shape[0] \
            or bbox[3] < 0 or bbox[4] < 0:
        return is_overlap

    x_tl = bbox[1] if bbox[1] > 0 else 0
    y_tl = bbox[2] if bbox[2] > 0 else 0
    x_br = bbox[3] if bbox[3] < mask.shape[1] else mask.shape[1] - 1
    y_br = bbox[4] if bbox[4] < mask.shape[0] else mask.shape[0] - 1
    vertexs = [[x_tl, y_tl], [x_tl, y_br], [x_br, y_tl], [x_br, y_br]]
    for v in vertexs:
        (g, b, r) = mask[v[1], v[0]]
        if (g, b, r) > (128, 128, 128):
            is_overlap = True
            return is_overlap

    return is_overlap

def is_same_direction(traj1, traj2, angle_thr):
    vec1 = np.array([traj1[-1][0] - traj1[0][0], traj1[-1][1] - traj1[0][1]])
    vec2 = np.array([traj2[-1][0] - traj2[0][0], traj2[-1][1] - traj2[0][1]])
    L1 = np.sqrt(vec1.dot(vec1))
    L2 = np.sqrt(vec2.dot(vec2))
    if L1 == 0 or L2 == 0:
        return False
    cos = vec1.dot(vec2)/(L1*L2)
    angle = np.arccos(cos) * 360/(2*np.pi)
    if angle < angle_thr:
        return True
    else:
        return False

def calc_angle(vec1, vec2):
    vec1 = np.array([traj1[-1][0] - traj1[-5][0], traj1[-1][1] - traj1[-5][1]])
    vec2 = np.array([traj2[-1][0] - traj2[-5][0], traj2[-1][1] - traj2[-5][1]])
    L1 = np.sqrt(vec1.dot(vec1))
    L2 = np.sqrt(vec2.dot(vec2))
    if L1 == 0 or L2 == 0:
        return 90
    cos = vec1.dot(vec2)/(L1*L2)
    if cos > 1:
        return 90
    angle = np.arccos(cos) * 360/(2*np.pi)
    return angle

def count_video(data_root, video_name, save_root):
    '''
    data_root: the path which contains (1)the track_results(2)masks(3)AIC20_track1(4)cam-configs
    video_name: the video to count
    save_root: the root path to save counting results
    '''
    #get cam_name related index
    #for datasetB, you shold change the path for video_list_id.txt 
    video_list_file = os.path.join(data_root, 'AIC20_track1/Dataset_A/list_video_id.txt')
    if not os.path.exists(video_list_file):
        print('no list_video_id.txt found! ')
        return
        
    with open(video_list_file, 'r') as fl:
        lines = [line.strip('\n').split(' ') for line in fl]
        for line in lines:
            if line[1][:-4] != video_name:
                continue
            else:
                video_idx = line[0]

    # load movements tipical trajs
    cam_name = 'cam_' + video_name.split('_')[1]
    cam_conf = os.path.join(data_root, 'cam-configs', cam_name+'.json')
    tipical_trajs = {}
    with open(cam_conf, 'r') as fc:
        movements = json.load(fc)
        for movement_id, movement_info in movements.items():
            try:
                tracklets = movement_info['tracklets']
                tipical_trajs[movement_id] = tracklets
            except:
                print('cam config error!')
                return

    #read mask image
    cam_mask = os.path.join(data_root, 'mask', video_name+'.jpg')
    mask = cv2.imread(cam_mask)
    h, w, c = mask.shape
    #load tracks
    tracks = {}
    #path to track results path
    track_file = os.path.join(data_root, 'track_results', video_name+'.txt')
    with open(track_file, 'r') as ft:
        lines = [line.strip('\n').split(' ') for line in ft]
        for line in lines:
            frameid = int(line[0])
            trackid = int(line[1])
            x1 = int(float(line[2]))
            y1 = int(float(line[3]))
            x2 = int(float(line[4]))
            y2 = int(float(line[5]))
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            label = line[6]
            if trackid in tracks:
                tracks[trackid]['endframe'] = frameid
                tracks[trackid]['bbox'].append([frameid, x1, y1, x2, y2, label])
                tracks[trackid]['tracklet'].append([cx, cy])
            else:
                tracks[trackid] = {'startframe' : frameid,
                                   'endframe' : frameid,
                                   'bbox' : [[frameid, x1, y1, x2, y2, label]],
                                   'tracklet' : [[cx, cy]]}

    #split tracklets
    tracks_end_in_roi, tracks_start_in_roi, tracks_too_short = check_tracks_with_roi(tracks, mask)

    trackids = sorted([k for k in tracks.keys()])
    #save count results
    os.makedirs(save_root, exist_ok=True)
    savefile = os.path.join(save_root, video_name+'.txt')
    dst_out = open(savefile, 'w')

    #start counting
    dist_thr = 300
    angle_thr = 30
    min_length = 10
    results = []
    for trackid in trackids:
        if len(tracks[trackid]['tracklet']) < min_length:
            continue
        track_traj = tracks[trackid]['tracklet']
        #calc hausdorff dist with tipical trajs, assign the movement with the min dist
        all_dists_dict = {k: float('inf') for k in tipical_trajs}
        for m_id, m_t in tipical_trajs.items():
            for t in m_t:
                tmp_dist = hausdorff_distance(np.array(track_traj), np.array(t), distance='euclidean')
                if tmp_dist < all_dists_dict[m_id]:
                    all_dists_dict[m_id] = tmp_dist

        #check direction
        all_dists = sorted(all_dists_dict.items(), key=lambda k: k[1])
        min_idx, min_dist = None, dist_thr
        for i in range(0, len(all_dists)):
            m_id = all_dists[i][0]
            m_dist = all_dists[i][1]
            if m_dist >= dist_thr: #if min dist > dist_thr, will not assign to any movement
                break
            else:
                if is_same_direction(track_traj, tipical_trajs[m_id][0], angle_thr): #check direction
                    min_idx = m_id
                    min_dist = m_dist
                    break #if match, end
                else:
                    continue #direction not matched, find next m_id

        #cam_13 14 will not use shape based method
        direct_match_videos = ['cam_13', 'cam_14']
        if cam_name not in direct_match_videos and min_idx == None and min_dist >= dist_thr:
            continue

        #spatial constrains
        #-----------------------------------------------------------------------------------
        #cam 1
        if cam_name == 'cam_1' and min_idx in ['movement_1', 'movement_2', 'movement_3'] and \
                len(track_traj) < 30:
            continue
        #is mv3
        if cam_name == 'cam_1' and min_idx != 'movement_3':
            cx, cy = track_traj[0]
            if cx > w * 0.5 and cy > h * 0.3:
                min_idx = 'movement_3'
        #is mv1 or 2 not 3
        if cam_name == 'cam_1' and min_idx == 'movement_3':
            cx, cy = track_traj[0]
            if cx < w * 0.5 or cy < h * 0.3:
                if is_same_direction(track_traj, tipical_trajs['movement_1'][0], 45):
                    min_idx = 'movement_1'
                elif is_same_direction(track_traj, tipical_trajs['movement_2'][0], 45):
                    min_idx = 'movement_2'

        #-----------------------------------------------------------------------------------
        #cam 2
        if cam_name == 'cam_2' and min_idx in ['movement_1', 'movement_2'] and \
                len(track_traj) < 30:
                    continue
        #-----------------------------------------------------------------------------------
        #cam_3
        #is mv4
        if cam_name == 'cam_3' and min_idx != 'movement_4':
            cx, cy = track_traj[0]
            if cx > w * 0.5 and cy > h * 0.5:
                min_idx = 'movement_4'
        #is mv 1 2 3 not mv4
        if cam_name == 'cam_3' and min_idx == 'movement_4':
            cx, cy = track_traj[0]
            if cx < w * 0.5 or cy < h * 0.5:
                if is_same_direction(track_traj, tipical_trajs['movement_1'][0], 45):
                    min_idx = 'movement_1'
                elif is_same_direction(track_traj, tipical_trajs['movement_2'][0], 45):
                    min_idx = 'movement_2'
                elif is_same_direction(track_traj, tipical_trajs['movement_3'][0], 45):
                    min_idx = 'movement_3'
        #-----------------------------------------------------------------------------------
        #cam4 5
        # is mv5 not mv 1 12
        if cam_name in ["cam_4", 'cam_5'] and min_idx in ["movement_1", 'movement_12']:
            cx, cy = track_traj[0]
            if cx > 300:
                if all_dists_dict['movement_5'] < dist_thr:
                    min_idx = 'movement_5'
                else:
                    continue
        #is mv8 not mv4
        if cam_name in ['cam_4', 'cam_5'] and min_idx == 'movement_4':
            cx, cy = track_traj[0]
            if cx > 1000:
                if all_dist_dict['movement_8'] < dist_thr:
                    min_idx = 'movement_8'
                else:
                    continue
        #is mv5 not mv4
        if cam_name in ['cam_4', 'cam_5'] and min_idx == 'movement_4':
            #print(trackid)
            cx, cy = track_traj[-1]
            if cx < 100:
                if all_dist_dict['movement_5'] < dist_thr:
                    min_idx = 'movement_5'
                else:
                    continue
        if cam_name in ['cam_4', 'cam_5'] and min_idx in ['movement_3'] and len(track_traj) < 30:
            continue
        if cam_name in ['cam_4', 'cam_5'] and min_idx in ['movement_5'] and \
                trackid in tracks_start_in_roi:
            continue

        #------------------------------------------------------------------------------------
        if cam_name == 'cam_7' and trackid in tracks_end_in_roi:
            continue

        #------------------------------------------------------------------------------------
        #cam 10 11
        if cam_name in ['cam_10', 'cam_11'] and min_idx == 'movement_1':
            if track_traj[0][0] > w * 0.5:
                continue
        if cam_name in ['cam_10', 'cam_11'] and min_idx == 'movement_3':
            if track_traj[-1][0] < 200:
                continue
        #------------------------------------------------------------------------------------
        #cam12
        if cam_name == 'cam_12' and min_idx in ['movement_1', 'movement_2']:
            if track_traj[0][0] > w * 0.6 or track_traj[0][0] < 100 or track_traj[0][1] > h * 0.5:
                continue
            if len(track_traj) < 30:
                continue
            #if min_idx == 'movement_2' and track_traj[-1][0] and track_traj[-1][1] < 600:
            #    print(trackid, track_traj[-1])
        #------------------------------------------------------------------------------------
        #cam13
        if cam_name == 'cam_13':
            if track_traj[0][1] > 374 or track_traj[0][0] > w * 0.7 or track_traj[0][1] > h * 0.8:
                continue
            if track_traj[-1][0] < w * 0.7 and track_traj[-1][1] > h * 0.8:
                min_idx = 'movement_2'
            elif track_traj[-1][0] > w * 0.7 and track_traj[-1][1] > h * 0.5:
                min_idx = 'movement_3'
            elif track_traj[-1][0] < w * 0.3 and track_traj[-1][1] < h * 0.8:
                min_idx = 'movement_1'
            else:
                continue
        #------------------------------------------------------------------------------------
        #cam14
        if cam_name == 'cam_14':
            if track_traj[-1][0] <= w * 0.5 and is_same_direction(track_traj,
                    tipical_trajs['movement_1'][0], 45):
                min_idx = 'movement_1'
            elif track_traj[-1][0] > w * 0.5 and is_same_direction(track_traj, tipical_trajs['movement_2'][0], 45):
                min_idx = 'movement_2'
            elif abs(track_traj[-1][0] - track_traj[0][0]) < 100:
            # for id switch in remote area which lead to the trackid go back 
                pass
            else:
                continue
        #------------------------------------------------------------------------------------
        if cam_name == 'cam_15':
            #remove still vehicle
            track_w = tracks[trackid]['bbox'][0][3] - tracks[trackid]['bbox'][0][1]
            if abs(track_traj[-1][0] - track_traj[0][0]) < 2*track_w:
                continue
            if track_traj[0][0] < w * 0.5:
                min_idx = 'movement_1'
            elif track_traj[0][0] > w * 0.7:
                min_idx = 'movement_2'
            else:
                continue
        #------------------------------------------------------------------------------------
        #cam_20
        if cam_name in ['cam_19', 'cam_20']:
            #remove still vehicle
            track_h = tracks[trackid]['bbox'][0][4] - tracks[trackid]['bbox'][0][2]
            if abs(track_traj[-1][1] - track_traj[0][1]) < track_h:
                continue

        #save counting results
        mv_idx = min_idx.split('_')[1]
        #get last frameid in roi
        bboxes = tracks[trackid]['bbox']
        bboxes.sort(key=lambda x: x[0])

        dst_frame = bboxes[0][0]
        last_bbox = bboxes[-1]
        if check_bbox_overlap_with_roi(last_bbox, mask) == True:
            dst_frame = last_bbox[0]
        else:
            for i in range(len(bboxes) - 2, 0, -1):
                bbox = bboxes[i]
                if check_bbox_overlap_with_roi(bbox, mask) == True:
                    dst_frame = bbox[0]
                    break
                else:
                    continue

        track_types = [k[5] for k in bboxes]
        track_type = max(track_types, key=track_types.count)

        if track_type == 'car':
            cls_id = 1
        else:
            cls_id = 2
        results.append([video_idx, dst_frame, mv_idx, cls_id])
    #save
    results.sort(key=lambda x: (x[1], x[2]))
    for res in results:
        res_str = " ".join(str(k) for k in res)
        print(res_str, file=dst_out)
    dst_out.close()
    print('vehicle counting done.')


if __name__ == '__main__':
    data_root = '../'
    save_root = './vehicle_counting_results'
    #video_list = os.listdir(os.path.join(data_root, 'imageset'))
    video_list = [sys.argv[1]]

    for video_name in video_list:
        print('start to counting video %s ... ' % video_name)
        count_video(data_root, video_name, save_root)

#Top 2: 

Method Used: Tracktor + Movement Classification + Vehicle Type Classification

#Tracking
Tracking without bell and whistles
[link text](https://github.com/phil-bergmann/tracking_wo_bnw)

Google Colab Tutorials:

[link text](https://colab.research.google.com/drive/1oe_vdQQUMSG3XHp1ki7kiE3HA2fSXkO-)

In [ ]:
from collections import deque

import numpy as np
import torch

from intersection_class import counter_classification

class Countor:
    """The main tracking file, here is where magic happens."""

    def __init__(self, obj_detect, calibration, device):
        
        self.device = device
        
        # area minimal
        obj_detect.det_min_area = calibration.calibration['detection_min_area_val']
        obj_detect.tck_min_area = calibration.calibration['tracks_min_area_val']
        
        # area maximal
        obj_detect.det_max_area = calibration.calibration['detection_max_area_val']
        
        # thresh score
        #print(self.roi_heads.score_thresh)
        obj_detect.det_score_thresh = calibration.calibration['detection_object_thresh']
        obj_detect.tck_score_thresh = calibration.calibration['tracks_object_thresh']
        
        # nms thresh
        #print(self.roi_heads.nms_thresh)
        obj_detect.det_nms_thresh = calibration.calibration['detection_nms_thresh']
        obj_detect.tck_nms_thresh = calibration.calibration['tracks_nms_thresh']
        #self.empty_var = torch.empty(0, device=device)
        
        # porcentage ROI in
        obj_detect.det_min_ROI_in = calibration.calibration['detection_ROI_in']
        obj_detect.tck_min_ROI_in = calibration.calibration['tracks_ROI_in']
                
        # networks
        self.obj_detect = obj_detect
        
        # ROI image
        self.ROI_tensor = calibration.ROI_tensor.to(device)
        
        # Movement classification
        self.Movement_classification = counter_classification(calibration.movements)
        
        # Countor
        self.min_length_trajectory = calibration.calibration['min_length_trajectory']
        
        # motion model parameters
        self.motion_model_cfg  = calibration.calibration['motion_model']
      
        # set the values to 
        self.reset()

    def reset(self, hard=True):
        self.tracks = []
        self.inactive_tracks = []

        if hard:
            self.track_num = 0
            self.results = {}
            self.countor_restults = {}
            self.im_index = 0

    def tracks_to_inactive(self, tracks):
        self.tracks = [t for t in self.tracks if t not in tracks]
        for t in tracks:
            t.pos = t.last_pos[-1]          
            vector   = t.get_vector().cpu().numpy()
            movement = self.Movement_classification.classify(vector[:-1])
            self.countor_restults[t.id.cpu().numpy().item()] =np.concatenate([np.array([self.im_index]).astype(int),movement,vector])
        self.inactive_tracks += tracks

    def add(self, new_det_pos, new_det_scores):
        """Initializes new Track objects and saves them."""
        num_new = new_det_pos.size(0)
        for i in range(num_new):
            self.tracks.append(Track(
                new_det_pos[i].view(1, -1),
                new_det_scores[i],
                torch.tensor([self.track_num + i]).cuda(),
                self.motion_model_cfg['n_steps'] if self.motion_model_cfg['n_steps'] > 0 else 1
            ))
        self.track_num += num_new
    
    def get_pos(self):
        """Get the positions of all active tracks."""
        if len(self.tracks) == 1:
            pos = self.tracks[0].pos
            ids = self.tracks[0].id
        elif len(self.tracks) > 1:
            pos = torch.cat([t.pos for t in self.tracks], 0)
            ids = torch.cat([t.id for t in self.tracks], 0)
        else:
            pos = torch.empty(0, device=self.device)
            ids = torch.empty(0, device=self.device)
        return pos,ids

    def motion_step(self, track):
        """Updates the given track's position by one step based on track.last_v"""
        if self.motion_model_cfg['center_only']:
            center_new = get_center(track.pos) + track.last_v
            track.pos = make_pos(*center_new, get_width(track.pos), get_height(track.pos)).to(self.device)
        else:
            track.pos = track.pos + track.last_v

    def motion(self):
        """Applies a simple linear motion model that considers the last n_steps steps."""
        for t in self.tracks:
            last_pos = list(t.last_pos)

            # avg velocity between each pair of consecutive positions in t.last_pos
            if self.motion_model_cfg['center_only']:
                vs = [get_center(p2).to(self.device) - get_center(p1).to(self.device) for p1, p2 in zip(last_pos, last_pos[1:])]
            else:
                vs = [p2 - p1 for p1, p2 in zip(last_pos, last_pos[1:])]

            t.last_v = torch.stack(vs).mean(dim=0)
            self.motion_step(t)

                    
    def step(self, image):
        """This function should be called every timestep to perform tracking with a blob
        containing the image information.
        """
        for t in self.tracks:
            # add current position to last_pos list
            t.last_pos.append(t.pos.clone())

        # apply motion model
        if len(self.tracks):
            # apply motion model
            if self.motion_model_cfg['enabled']:
                self.motion()
                self.tracks = [t for t in self.tracks if t.has_positive_area()]
                
        pos, ids =self.get_pos()
        
        # Run the neural network
        det_boxes, det_scores, det_labels, tck_boxes, tck_scores, tck_labels, tck_ids = self.obj_detect(image, 
                                                                                                        [pos],
                                                                                                        [ids],
                                                                                                 [self.ROI_tensor])

        det_boxes = det_boxes[0].detach()
        det_scores = det_scores[0].detach()
        det_labels = det_labels[0].detach()
        tck_boxes = tck_boxes[0].detach()
        tck_scores = tck_scores[0].detach()
        tck_labels = tck_labels[0].detach()
        tck_ids = tck_ids[0].detach()
        
        # update position and score for the current tracks
        tracks_to_inactive_list = []
        if len(self.tracks):
            for i in range(len(self.tracks) - 1, -1, -1):
                if self.tracks[i].id in tck_ids:
                    index = tck_ids == self.tracks[i].id
                    index = index.nonzero().squeeze(1)
                    
                    self.tracks[i].pos   = tck_boxes[index].view(1, -1)
                    self.tracks[i].score = tck_scores[index]
                else:
                    # if the index is not in the ids set to inactive
                    tracks_to_inactive_list.append(self.tracks[i])

        # set the intective tracks  
        self.tracks_to_inactive(tracks_to_inactive_list)
        
        # Create new tracks
        if det_boxes.nelement() > 0:
            self.add(det_boxes, det_scores)
                

        ####################
        # Generate Results #
        ####################
        for t in self.tracks:
            if t.id.cpu().numpy().item() not in self.results.keys():
                self.results[t.id.cpu().numpy().item()] = {}
            self.results[t.id.cpu().numpy().item()][self.im_index] = np.concatenate([t.pos[0].cpu().numpy(), np.array([t.score.cpu()])])
        
        ####################
        # Prepare next frame #
        ####################
        self.im_index += 1
        self.last_image = image[0]   
        
    def get_results(self):
        return self.results


class Track(object):
    """This class contains all necessary for every individual track."""

    def __init__(self, pos, score, track_id, mm_steps):
        self.id = track_id
        self.pos = pos
        self.init_pos = pos
        self.score = score
        self.ims = deque([])
        self.last_pos = deque([pos.clone()], maxlen=mm_steps + 1)
        self.last_v = torch.Tensor([])


    def has_positive_area(self):
        return self.pos[0, 2] > self.pos[0, 0] and self.pos[0, 3] > self.pos[0, 1]
        
    def get_vector(self):
        x1y1 = get_center(self.init_pos)
        x2y2 = get_center(self.pos)
        dist = torch.dist(x1y1, x2y2,2).unsqueeze(0)
        return torch.cat([x1y1,x2y2,dist])

    
def get_center(pos):
    x1 = pos[0, 0]
    y1 = pos[0, 1]
    x2 = pos[0, 2]
    y2 = pos[0, 3]
    return torch.Tensor([(x2 + x1) / 2, (y2 + y1) / 2]).cuda()


def get_width(pos):
    return pos[0, 2] - pos[0, 0]


def get_height(pos):
    return pos[0, 3] - pos[0, 1]


def make_pos(cx, cy, width, height):
    return torch.Tensor([[
        cx - width / 2,
        cy - height / 2,
        cx + width / 2,
        cy + height / 2
    ]]).cuda()

#Countor Neural Network

In [ ]:
import torch
import torch.nn.functional as F

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.transform import resize_boxes
from torchvision.ops.boxes import clip_boxes_to_image, nms

class Countor_NN(FasterRCNN):

    def __init__(self):
        backbone = resnet_fpn_backbone('resnet50', False)
        super(Countor_NN, self).__init__(backbone, 91)
        # get cars trucks and bus classes
        self.selected_classes = [3,6,8]
        
        # area minimal
        self.det_min_area = 100
        self.tck_min_area = 100
        
        # area maximal
        self.det_max_area = 1000000
        
        # thresh score
        #print(self.roi_heads.score_thresh)
        self.det_score_thresh = 0.5
        self.tck_score_thresh = 0.05
        
        # nms thresh
        #print(self.roi_heads.nms_thresh)
        self.det_nms_thresh = 0.5
        self.tck_nms_thresh = 0.5
        #self.empty_var = torch.empty(0, device=device)
        
        # porcentage ROI in
        self.det_min_ROI_in = 0.3
        self.tck_min_ROI_in = 0.1
        
    def get_parameters(self):
        
        parameters = {}
        # area minimal
        parameters['det_min_area'] = self.det_min_area 
        parameters['tck_min_area'] = self.tck_min_area 
        
        # area maximal
        parameters['det_max_area'] = self.det_max_area 
        
        # thresh score
        #print(self.roi_heads.score_thresh)
        parameters['det_score_thresh'] = self.det_score_thresh 
        parameters['tck_score_thresh'] = self.tck_score_thresh 
        
        # nms thresh
        #print(self.roi_heads.nms_thresh)
        parameters['det_nms_thresh'] = self.det_nms_thresh 
        parameters['tck_nms_thresh'] = self.tck_nms_thresh 
        #self.empty_var = torch.empty(0, device=device)
        
        # porcentage ROI in
        parameters['det_min_ROI_in'] = self.det_min_ROI_in
        parameters['tck_min_ROI_in'] = self.tck_min_ROI_in
        
        return parameters
        
    @staticmethod
    def remove_small_boxes_area(boxes, min_size):
        
        # type: (Tensor, float)
        """
        Remove boxes which area is smaller than min_size.
        Arguments:
            boxes (Tensor[N, 4]): boxes in (x1, y1, x2, y2) format
            min_size (float): minimum size
        Returns:
            keep (Tensor[K]): indices of the boxes that have both sides
                larger than min_size
        """
        area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        keep = area >= min_size
        keep = keep.nonzero().squeeze(1)
        return keep

    @staticmethod
    def remove_big_boxes_area(boxes, max_size):
        
        # type: (Tensor, float)
        """
        Remove boxes which area is smaller than min_size.
        Arguments:
            boxes (Tensor[N, 4]): boxes in (x1, y1, x2, y2) format
            min_size (float): minimum size
        Returns:
            keep (Tensor[K]): indices of the boxes that have both sides
                larger than min_size
        """
        area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        keep = area <= max_size
        keep = keep.nonzero().squeeze(1)
        return keep
            
    @staticmethod
    def remove_boxes_out_roi(boxes, ROI_image, min_in_porcentage):
        
        # type: (Tensor, float)
        """
        Remove boxes which contains at least one side smaller than min_size.
        Arguments:
            boxes (Tensor[N, 4]): boxes in (x1, y1, x2, y2) format
            min_size (float): minimum size
        Returns:
            keep (Tensor[K]): indices of the boxes that have both sides
                larger than min_size
        """
        
        area_in =[ROI_image[box[1]:box[3],box[0]:box[2]].sum() for box in boxes.int()]
        area_in = torch.stack(area_in) 
        area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        porcentage_in = area_in/area.float()
        keep = porcentage_in >= min_in_porcentage
        keep = keep.nonzero().squeeze(1)
        return keep
    
    # detections roi head

    def detections_processing(self, images, features, ROI_images, tck_boxes, original_image_sizes):
        # rpn network
        det_proposals, proposal_losses = self.rpn(images, features)

        # roi heads to get boxes and scores
        det_box_features = self.roi_heads.box_roi_pool(features, det_proposals, images.image_sizes)
        det_box_features = self.roi_heads.box_head(det_box_features)
        det_class_logits, det_box_regression = self.roi_heads.box_predictor(det_box_features)
        det_boxes = self.roi_heads.box_coder.decode(det_box_regression, det_proposals)
        det_scores = F.softmax(det_class_logits, -1)


        boxes_per_image = [boxes_in_image.shape[0] for boxes_in_image in det_proposals]

        # this gets the max score and gives the label
        # this mean just for 3 classes get just one
        det_scores, det_labels = det_scores[:,self.selected_classes].max(1)
        det_boxes  = det_boxes[:,self.selected_classes]
        det_boxes = torch.cat([det_boxes[idx][i].unsqueeze(0)for idx, i in enumerate(det_labels)])

        # split the boxes scores and labels for each image for post processing
        det_boxes_list  = det_boxes.split(boxes_per_image, 0)
        det_scores_list = det_scores.split(boxes_per_image, 0)
        det_labels_list = det_labels.split(boxes_per_image, 0)

        det_all_boxes = []
        det_all_scores = []
        det_all_labels = []
        

        for boxes, scores, labels, ROI_image, tck_boxes_b, image_shape, original_im_shape in zip(det_boxes_list, 
                                                                         det_scores_list, 
                                                                         det_labels_list,
                                                                         ROI_images,
                                                                         tck_boxes,
                                                                         images.image_sizes, 
                                                                         original_image_sizes):

            boxes = clip_boxes_to_image(boxes, image_shape)

            # batch everything, by making every class prediction be a separate instance
            boxes = boxes.reshape(-1, 4)
            scores = scores.reshape(-1)
            labels = labels.reshape(-1)
            
            # remove low scoring boxes 
            inds = torch.nonzero(scores > self.det_score_thresh).squeeze(1)
            boxes, scores, labels = boxes[inds], scores[inds], labels[inds]

            # remove small boxes
            keep = self.remove_small_boxes_area(boxes, min_size=self.det_min_area)
            boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
            
            # remove too big boxes
            #keep = self.remove_big_boxes_area(boxes, max_size=self.det_max_area)
            #boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
            
            # non-maximum suppression, independently done per class
            keep = nms(boxes, scores, self.det_nms_thresh)
            #boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
            
            # keep only topk scoring predictions
            keep = keep[:self.roi_heads.detections_per_img]
            boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
            
            boxes = resize_boxes(boxes, image_shape, original_im_shape)
            
            if boxes.nelement():
                keep = self.remove_boxes_out_roi(boxes, ROI_image, min_in_porcentage=self.det_min_ROI_in)
                boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
                
            # filter detections in tracks  
            for tck_box in tck_boxes_b:
                temp_boxes  = torch.cat([tck_box.unsqueeze(0), boxes])
                temp_scores = torch.cat([torch.tensor([2.0]).to(boxes.device), scores])
                keep   = nms(temp_boxes, temp_scores, self.det_nms_thresh)
                keep   = keep[torch.ge(keep, 1)] - 1
                boxes, scores, labels = boxes[keep], scores[keep], labels[keep]
                if keep.nelement() == 0:
                    break

            
            det_all_boxes.append(boxes)
            det_all_scores.append(scores)
            det_all_labels.append(labels)

        return det_all_boxes, det_all_scores, det_all_labels


    def tracking_processing(self,boxes, boxes_ids, images , features, ROI_images, original_image_sizes):
        device = list(self.parameters())[0].device
        # resize all the given box to be aligned
        tck_proposals = [resize_boxes(box, or_size, size) for box, or_size, size in zip(boxes, 
                                                                                        original_image_sizes, 
                                                                                        images.image_sizes) if box.nelement()]
        
        tck_all_boxes  = []
        tck_all_scores = []
        tck_all_labels = []
        tck_all_ids = []
        
        boxes_per_image = [boxes_in_image.shape[0] for boxes_in_image in tck_proposals]
        
        if tck_proposals:
            tck_box_features = self.roi_heads.box_roi_pool(features, tck_proposals, images.image_sizes)
            tck_box_features = self.roi_heads.box_head(tck_box_features)
            tck_class_logits, tck_box_regression = self.roi_heads.box_predictor(tck_box_features)
            tck_boxes = self.roi_heads.box_coder.decode(tck_box_regression, tck_proposals)
            tck_scores = F.softmax(tck_class_logits, -1)

            tck_scores, tck_labels = tck_scores[:,self.selected_classes].max(1)
            tck_boxes = tck_boxes[:,self.selected_classes]
            tck_boxes = torch.cat([tck_boxes[idx][i].unsqueeze(0)for idx, i in enumerate(tck_labels)])

            tck_boxes_list  = tck_boxes.split(boxes_per_image, 0)
            tck_scores_list = tck_scores.split(boxes_per_image, 0)
            tck_labels_list = tck_labels.split(boxes_per_image, 0)
            
            for boxes, scores, labels, box_ids, ROI_image, image_shape, original_im_shape in zip(tck_boxes_list, 
                                                                         tck_scores_list, 
                                                                         tck_labels_list, 
                                                                         boxes_ids,
                                                                         ROI_images,
                                                                         images.image_sizes, 
                                                                         original_image_sizes):
            
                boxes = clip_boxes_to_image(boxes, image_shape)

                # batch everything, by making every class prediction be a separate instance
                boxes = boxes.reshape(-1, 4)
                scores = scores.reshape(-1)
                labels = labels.reshape(-1)
                box_ids = box_ids.reshape(-1)

                # remove low scoring boxes 
                keep    = torch.nonzero(scores > self.tck_score_thresh).squeeze(1)
                boxes   = boxes[keep]
                scores  = scores[keep]
                labels  = labels[keep]
                box_ids = box_ids[keep]

                # remove small boxes
                keep    = self.remove_small_boxes_area(boxes, min_size=self.tck_min_area)
                boxes   = boxes[keep]
                scores  = scores[keep]
                labels  = labels[keep]
                box_ids = box_ids[keep]
                
                # non-maximum suppression, independently done per class
                keep    = nms(boxes, scores, self.tck_nms_thresh)
                boxes   = boxes[keep]
                scores  = scores[keep]
                labels  = labels[keep]
                box_ids = box_ids[keep]
                # keep only topk scoring predictions

                boxes = resize_boxes(boxes, image_shape, original_im_shape)
                
                if boxes.nelement():
                    
                    keep = self.remove_boxes_out_roi(boxes, ROI_image, min_in_porcentage=self.tck_min_ROI_in)
                    boxes   = boxes[keep]
                    scores  = scores[keep]
                    labels  = labels[keep]
                    box_ids = box_ids[keep]

                tck_all_boxes.append(boxes)
                tck_all_scores.append(scores)
                tck_all_labels.append(labels)
                tck_all_ids.append(box_ids)
        else:
            tck_all_boxes.append(torch.empty(0, device=device))
            tck_all_scores.append(torch.empty(0, device=device))
            tck_all_labels.append(torch.empty(0, device=device))
            tck_all_ids.append(torch.empty(0, device=device))
        
        return tck_all_boxes,tck_all_scores,tck_all_labels, tck_all_ids
           
    def forward(self, images, boxes, boxes_ids, ROI_images):
        # inputs to device
        device = list(self.parameters())[0].device
        
        # get the original image size
        original_image_sizes = [img.shape[-2:] for img in images]
        
        targets = None
        # transform the images with the standart 
        images, targets = self.transform(images,targets)
        # get the features form the transformd images
        features = self.backbone(images.tensors)
        if isinstance(features, torch.Tensor):
            features = OrderedDict([(0, features)])
            
        # get the list of tracks
        tck_boxes, tck_scores, tck_labels, tck_ids = self.tracking_processing(boxes, 
                                                                              boxes_ids, 
                                                                              images, 
                                                                              features, 
                                                                              ROI_images, 
                                                                              original_image_sizes)
            
        # get the list of detections
        det_boxes, det_scores, det_labels = self.detections_processing(images, 
                                                                       features, 
                                                                       ROI_images, 
                                                                       tck_boxes, 
                                                                       original_image_sizes)
        
        return det_boxes, det_scores, det_labels, tck_boxes, tck_scores, tck_labels, tck_ids



#Counting Classification: 

In [ ]:
import pandas as pd
import numpy as np

def ccw2(A,B,C):
    return (C[:,1]-A[:,1])*(B[:,0]-A[:,0]) > (B[:,1]-A[:,1])*(C[:,0]-A[:,0])

def intersect2(A,B):
    o1 = ccw2(A[:,0:2],B[:,0:2],B[:,2:4]) 
    o2 = ccw2(A[:,2:4],B[:,0:2],B[:,2:4]) 
    o3 = ccw2(A[:,0:2],A[:,2:4],B[:,0:2]) 
    o4 = ccw2(A[:,0:2],A[:,2:4],B[:,2:4])
    return np.logical_and((o1 != o2),(o3 != o4)), o1

class counter_classification:
    def __init__(self, df_mov):

        # get info numpy to optimize running time
        self.calib_lines = df_mov.values[:,1:5].astype(int)
        self.bool_guide  = df_mov.values[:,5].astype(bool)       
        self.ids_movements = np.unique(df_mov.values[:,0]).astype(int)
        self.direction_ref = df_mov.values[:,6].astype(bool)
        self.bool_nan_dir = df_mov.values[:,6]!=df_mov.values[:,6]
        # create a checklist to see if the conditions were met
        self.check_list = np.zeros((self.ids_movements.shape[0],self.bool_guide .shape[0]), dtype=bool)
        for i,id in enumerate(self.ids_movements):
            self.check_list[i] = df_mov.values[:,0]==id

    def classify(self,vector):

        out = np.zeros((self.ids_movements.shape[0]),dtype=bool)
        # create matrix with the input vector
        vector2intersect = np.zeros_like(self.calib_lines,dtype=int)
        vector2intersect[:] = vector
        # intersect the vector with every calib line
        res_inter, direction = intersect2(self.calib_lines,vector2intersect)
        # check if the lines should intersec or not
        bool_res = np.equal(res_inter, self.bool_guide)
        # check if the detected cross is in the good direction
        bool_direction = np.equal(self.direction_ref, direction)
        # if we dont care put the dirction to true it to true
        bool_direction[self.bool_nan_dir]=True
        # check all the crossing conditions
        for i,check in enumerate(self.check_list):
            out[i]= bool_res[check].all() and bool_direction[check].all()
        # create the output
        if self.ids_movements[out].size == 0:
            output = np.array([-1])
        elif self.ids_movements[out].size > 1:
            output = np.array([0])
        else:
            output = self.ids_movements[out]

        return output

#Top 3: 


#Detection: Mask RCNN

Google Colab Tutorials:

[link text](https://github.com/matterport/Mask_RCNN/blob/master/samples/demo.ipynb)

![alt text](https://raw.githubusercontent.com/matterport/Mask_RCNN/master/assets/street.png)

#Tracking: Toward-Realtime-MOT

[link text](https://github.com/Lijun-Yu/zero_virus/tree/master/tracker)